In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/investments/sti_return.csv
/kaggle/input/stock-prices/sg_stock_prices.csv
/kaggle/input/ffm-factors/ffm_factors-1.5.csv


In [2]:
df = pd.read_csv('/kaggle/input/stock-prices/sg_stock_prices.csv')
df.head()

,Company Name,GVKEY,Year,Price,Return,Return (t-1),M/B,Size,Score
0,GREAT EASTERN HOLDINGS LTD,15565,2006,17.00,0.156,.,2.74,42.03,4.42
1,GREAT EASTERN HOLDINGS LTD,15565,2007,16.60,-0.024,0.156,2.39,46.52,1.95
2,GREAT EASTERN HOLDINGS LTD,15565,2008,9.06,-0.454,-0.024,1.42,44.03,7.08
3,GREAT EASTERN HOLDINGS LTD,15565,2009,13.54,0.494,-0.454,1.80,48.53,4.16
4,GREAT EASTERN HOLDINGS LTD,15565,2010,15.62,0.154,0.494,1.84,53.37,6.23


In [3]:
df = df.sort_values(by=['GVKEY','Year'])
df.head()

,Company Name,GVKEY,Year,Price,Return,Return (t-1),M/B,Size,Score
0,GREAT EASTERN HOLDINGS LTD,15565,2006,17.00,0.156,.,2.74,42.03,4.42
1,GREAT EASTERN HOLDINGS LTD,15565,2007,16.60,-0.024,0.156,2.39,46.52,1.95
2,GREAT EASTERN HOLDINGS LTD,15565,2008,9.06,-0.454,-0.024,1.42,44.03,7.08
3,GREAT EASTERN HOLDINGS LTD,15565,2009,13.54,0.494,-0.454,1.80,48.53,4.16
4,GREAT EASTERN HOLDINGS LTD,15565,2010,15.62,0.154,0.494,1.84,53.37,6.23


In [4]:
x=1
df['momentum'] = df.groupby('GVKEY')['Price'].transform(lambda v: v - v.shift(x))

In [5]:
df

,Company Name,GVKEY,Year,Price,Return,Return (t-1),M/B,Size,Score,momentum
0,GREAT EASTERN HOLDINGS LTD,15565,2006,17.00,0.156,.,2.74,42.03,4.42,NaN
1,GREAT EASTERN HOLDINGS LTD,15565,2007,16.60,-0.024,0.156,2.39,46.52,1.95,-0.40
2,GREAT EASTERN HOLDINGS LTD,15565,2008,9.06,-0.454,-0.024,1.42,44.03,7.08,-7.54
3,GREAT EASTERN HOLDINGS LTD,15565,2009,13.54,0.494,-0.454,1.80,48.53,4.16,4.48
4,GREAT EASTERN HOLDINGS LTD,15565,2010,15.62,0.154,0.494,1.84,53.37,6.23,2.08
...,...,...,...,...,...,...,...,...,...,...
9233,OTS HOLDINGS LIMITED,350396,2022,0.18,-0.288,.,1.30,0.04,1.89,NaN
9234,AUDIENCE ANALYTICS LTD,350729,2022,0.28,0.273,.,2.94,0.02,9.18,NaN
9235,DAIWA HOUSE LOGISTICS TRUST,351449,2022,0.64,-0.206,.,0.75,1.18,2.94,NaN
9236,MOOREAST HOLDINGS LTD,351488,2022,0.16,-0.069,.,1.81,0.07,5.43,NaN


In [6]:
# Assign quintile ranks based on the momentum score
df['momentum_rank'] = pd.qcut(df['momentum'], 5, labels=False, duplicates='drop') + 1

# Calculate average return per momentum quintile annually
momentum_portfolio_returns = df.groupby(['Year', 'momentum_rank'])['Return'].mean().unstack()

In [7]:

momentum_portfolio_returns = momentum_portfolio_returns.iloc[1:]
momentum_portfolio_returns.reset_index(inplace=True)

In [8]:
print(momentum_portfolio_returns)

momentum_rank  Year       1.0       2.0       3.0       4.0       5.0
0              2007 -0.297333 -0.141500  0.010514  0.419000  0.995571
1              2008 -0.616854 -0.514806 -0.314333  0.157833  1.727000
2              2009 -0.363933 -0.254036  0.026310  0.676545  1.590011
3              2010 -0.291878 -0.214920 -0.098314  0.239690  0.605541
4              2011 -0.386479 -0.295754 -0.202115  0.262475  0.506500
5              2012 -0.339121 -0.229712 -0.020711  0.547548  0.975050
6              2013 -0.293851 -0.165790 -0.090291  0.923441  0.563218
7              2014 -0.356000 -0.254095 -0.112444  0.228434  0.550469
8              2015 -0.376941 -0.231759 -0.198134  0.294386  0.666625
9              2016 -0.400765 -0.236987 -0.132689  0.219110  0.731564
10             2017 -0.344979 -0.240036 -0.072946  0.503810  0.734839
11             2018 -0.329143 -0.272747 -0.204168  0.390162  0.669231
12             2019 -0.368622 -0.254823 -0.105770  0.251726  0.501391
13             2020 

In [9]:
mom_factor = pd. DataFrame()
mom_factor['Year'] = [i for i in range(2007,2023)]
#mom_factor['Year'] = momentum_portfolio_returns['Year']

mom_factor['MOM'] = ((momentum_portfolio_returns[5]+momentum_portfolio_returns[4])/2) - ((momentum_portfolio_returns[1]+momentum_portfolio_returns[2])/2)

In [10]:
mom_factor


,Year,MOM
0,2007,0.926702
1,2008,1.508247
2,2009,1.442262
3,2010,0.676015
4,2011,0.725604
5,2012,1.045715
6,2013,0.973151
7,2014,0.694499
8,2015,0.784856
9,2016,0.794213


In [11]:
factors = pd.read_csv('/kaggle/input/ffm-factors/ffm_factors-1.5.csv')
factors
factors = factors[factors['Year']>2006]
#factors.drop(columns=['index'])
factors.reset_index(inplace=True)

In [12]:
import statsmodels.api as sm

def run_factor_model(portfolio_returns, factors, mom_factor):
    
    results = {}
    print(mom_factor)
    factors['mom'] = mom_factor['MOM']
    print(factors)
    for portfolio in portfolio_returns.columns:
        if portfolio != 'Year':
            y = portfolio_returns[portfolio].dropna()
            X = factors[['market_excess', 'smb', 'hml', 'mom']]
            #print(X)
            X = sm.add_constant(X)  
            model = sm.OLS(y, X).fit()
            results[portfolio] = model.params
            print(model.summary())

    return pd.DataFrame(results)

# Assuming 'factors' contains your market, SMB, and HML data
regression_results_1_6 = run_factor_model(momentum_portfolio_returns, factors, mom_factor)

    Year       MOM
0   2007  0.926702
1   2008  1.508247
2   2009  1.442262
3   2010  0.676015
4   2011  0.725604
5   2012  1.045715
6   2013  0.973151
7   2014  0.694499
8   2015  0.784856
9   2016  0.794213
10  2017  0.911832
11  2018  0.830642
12  2019  0.688281
13  2020  1.374611
14  2021  0.786675
15  2022  0.680837
    index  Unnamed: 0  Year       smb       hml  market_excess       mom
0       1           7  2007  0.334451  0.264053           0.17  0.926702
1       2           8  2008 -0.013658  0.086266          -0.49  1.508247
2       3           9  2009 -0.067836  0.959928           0.64  1.442262
3       4          10  2010 -0.066922  0.222187           0.10  0.676015
4       5          11  2011 -0.044865  0.137002          -0.17  0.725604
5       6          12  2012  0.042415  0.630474           0.20  1.045715
6       7          13  2013  0.010752  0.720292           0.00  0.973151
7       8          14  2014 -0.006299 -0.068327           0.06  0.694499
8       9          1

/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=1

In [13]:
regression_results_1_6

,1.0,2.0,3.0,4.0,5.0
const,-0.195233,-0.090531,-0.109770,0.237288,-0.523052
market_excess,0.061573,0.084929,0.262098,-0.081958,0.228460
smb,0.184867,0.322725,0.325234,0.186080,0.321512
hml,0.174888,0.163766,0.089126,0.669598,-0.330944
mom,-0.244020,-0.246431,-0.052356,-0.076747,1.586296
